<a href="https://colab.research.google.com/github/Shohei496/Deep-met/blob/circleci-project-setup/step1%E3%82%A6%E3%82%A4%E3%83%8A%E3%83%BC%E3%83%95%E3%82%A3%E3%83%AB%E3%82%BF_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#0
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#1
!sudo apt-get install libportaudio2
!pip install sounddevice

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 1s (67.7 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 156210 file

In [ ]:
#2
import wave as wave
import numpy as np
import scipy.signal as sp
import sounddevice as sd
import librosa
import librosa.display
import os
import pandas as pd

In [ ]:
#3

#####データの種類
Type = 0      ###### 学習データをフィルタリングする場合は0を入力,　テストデータの場合は1を記入

In [ ]:
#4

base_dir = "drive/MyDrive/"
esc_dir = os.path.join(base_dir, "data")
audio_dir = os.path.join(esc_dir, "audio/")

if Type < 1:
   meta_file = os.path.join(esc_dir, "meta/meta_data.csv")
else:
   meta_file = os.path.join(esc_dir, "meta/meta_data_test.csv")
#audio_dir2 = os.path.join(esc_dir, "audioa/")


In [ ]:
#5

# load metadata
meta_data = pd.read_csv(meta_file, encoding="shift-jis")
meta_data

,filename,target
0,10a1.wav のコピー,0
1,a2.wav のコピー,0
2,a3.wav のコピー,0
3,a4.wav のコピー,0
4,a5.wav のコピー,0
...,...,...
242,8:45.wav のコピー,3
243,8:46.wav のコピー,3
244,8:47.wav のコピー,3
245,8:48.wav のコピー,3


In [ ]:
#6

# get data size
data_size = meta_data.shape
print(data_size)

(247, 2)


In [ ]:
#7

x = list(meta_data.loc[:,"filename"])
y = list(meta_data.loc[:, "target"])

In [ ]:
#8

# load a sound data
def load_sound_data(audio_dir, file_name):
    file_path = os.path.join(audio_dir, file_name)
    wavs = wave.open(file_path)
    
    return wavs


In [ ]:
#9

def save_np_data(filename, x, y, aug=None, rates=None):

    np_targets = np.zeros(len(y))
    for i in range(len(y)):
              
        wav = load_sound_data(audio_dir,  x[i])
       
        
        #音声区間の長さを取る
        
        n_speech=wav.getnframes()

        #サンプリングレートを取る
        sampling_rate=wav.getframerate()
        
        #PCM形式の波形データを読み込み
        speech_signal=wav.readframes(wav.getnframes())

        #speech_signalを2バイトの数値列に変換
       
        speech_signal=np.frombuffer(speech_signal, dtype=np.int16)

        #雑音だけの区間のサンプル数を設定
        n_noise_only=40000

        #全体の長さ
        n_sample=n_noise_only+n_speech


        #短時間フーリエ変換を行う
        f,t,stft_data=sp.stft(speech_signal,fs=wav.getframerate(),window="hann",nperseg=512,noverlap=256)

        #入力信号の振幅を取得
        amp=np.abs(stft_data)

       #入力信号のパワーを取得
        input_power=np.power(amp,2.0)

        #雑音だけの区間のフレーム数
        n_noise_only_frame=np.sum(t<(n_noise_only/sampling_rate))

        #ウィナーフィルタのパラメータ
        alpha=1 #デフォルト=1.0
        mu=1    #デフォルト=10

        #雑音のパワーを推定
        noise_power=np.mean(np.power(amp,2.0)[:,:n_noise_only_frame],axis=1,keepdims=True)

        #入力信号の音量の1%を下回らないようにする
        eps=0.01*input_power

        #出力信号の振幅を計算する
        processed_power=np.maximum(input_power-alpha*noise_power,eps)

        #比率
        wf_ratio= processed_power/(processed_power+mu*noise_power)


        #出力信号の振幅に入力信号の位相をかける
        processed_stft_data=wf_ratio*stft_data


        #時間領域の波形に戻す

        t,processed_data_post=sp.istft(processed_stft_data,fs=wav.getframerate(),window="hann",nperseg=512,noverlap=256)

        #2バイトのデータに変換
        processed_data_post=processed_data_post.astype(np.int16)


        #雑音消去
        processed_data_post=speech_signal-processed_data_post


        #waveファイルに書き込む
        wave_out = wave.open(audio_dir +'Filter/'+ x[i], 'w')
        
        #モノラル:1、ステレオ:2
        wave_out.setnchannels(1)

        #サンプルサイズ2byte
        wave_out.setsampwidth(2)

        #サンプリング周波数
        wave_out.setframerate(wav.getframerate())

        #データを書き込み

        wave_out.writeframes(processed_data_post)

        #ファイルを閉じる
        wave_out.close()


        #waveファイルを閉じる
        wav.close()
 
save_np_data("esc_melsp_test.npz", x,  y)
